## Dialectic GPT

The intent of this notebook is to arrange a dialogue between two chatbots (probably both openAI) who hold opposing positions on some issue.
This can then treated as either (or both):
1. A debate, in which an adjudicator determines the winner of the debate and explains why  the winner was chosen
2. Dialectic, in which case a synthesizer provedes a synthesis (with explanations) of the arguments.

Examples would be:
1. Which came first the chicken or the egg
2. Kantian vs Utilitarian ethicist
3. Star Trek vs Star Wars



In [1]:
import numpy as np
#import matplotlib.pyplot as plt

import os
import openai
import langchain

In [2]:
os.environ["OPENAI_API_KEY"] = open("/home/psg/openai_api_key.txt",'r').read()[:-1]
openai.api_key_path = "/home/psg/openai_api_key.txt"

In [3]:
from langchain.prompts.prompt import PromptTemplate
from langchain.llms import OpenAI
from langchain.chains import ChatVectorDBChain

In [7]:
def debateGPT(role1,role2,issue,no_rnds = 2,message_adjective = "concisely",print_debate=True, adjudicator=True,synthesizer=False,model="gpt-3.5-turbo"):
    '''Produces a dialogue between two chatbots (both openAI) who hold opposing positions on some issue. This can then treated as: 
    a debate, in which an adjudicator determines the winner of the debate and explains why the winner was chosen
    and/or a dialectic, in which case a synthesizer provides a synthesis (with explanations) of the arguments.'''
    # start
    init_statement = f"{message_adjective} state your position with respect to {issue}."
    respond_prompt1 = f"Consider and {message_adjective} respond to the points made by the {role2}."
    respond_prompt2 = f"Consider and {message_adjective} respond to the points made by the {role1}."

    template1 = f"Your are a {role1} who is debating with a {role2} on the question of: {issue}"
    template2 = f"Your are a {role2} who is debating with a {role1} on the question of: {issue}"

    if adjudicator:
        adjudicator_template = f"You are an adjudicator who is to assess the relative merits of the following arguments provided by {role1} and {role2} \
        and render a conclusion as to who's position on {issue} is superior. Give a definite conclusion with an explanation of your conclusion"
    else:
        adjudication = ""
    if synthesizer:
        synthesizer_template = f"You are an synthesizer who who considers the following arguments provided by {role1} and {role2} \
        and arrives at a position on {issue}. Give a description of your position and an explanation of how you arrived at your position"
    else:
        synthesis = ""
        
    init1 = openai.ChatCompletion.create(
      model=model,
      messages=[
        {"role": "user", "content": template1 + init_statement}
      ]

    )
    init2 = openai.ChatCompletion.create(
      model=model,
      messages=[
        {"role": "user", "content": template2 + init_statement}
      ]
    )
    position1 = role1 + ": " + init1.choices[0].message.content
    #print(position1)
    position2 = role2 + ": " + init2.choices[0].message.content
    #print(position2)
    #init_output1 = qa_chain(
    cum_arg = position1 + "\n\n" + position2
    #cum_arg2 = position2
    for rnd in range(no_rnds):
        #print(f"{role1}[{rnd}]:")
        resp1 = openai.ChatCompletion.create(
          model=model,
          messages=[
            {"role": "user", "content": template1 + cum_arg + respond_prompt1}
              ]
            )
        cum_arg = cum_arg + f"\n\n{role1}" + ": " + resp1.choices[0].message.content
        resp2 = openai.ChatCompletion.create(
          model=model,
          messages=[
            {"role": "user", "content": template2 + cum_arg + respond_prompt2}
              ]
            )
        cum_arg = cum_arg + f"\n\n{role2}" + ": " + resp2.choices[0].message.content
        #chat_history.append((question, result["answer"]))
        #print(f"{role1}[{rnd}]:" )
        #print(result["answer"]
    if adjudicator:
        adjudication = openai.ChatCompletion.create(
          model=model,
          messages=[
            {"role": "user", "content": adjudicator_template + cum_arg}
              ]
            )
    if synthesizer:
        synthesis = openai.ChatCompletion.create(
          model=model,
          messages=[
            {"role": "user", "content": synthesizer_template + cum_arg}
              ]
            )

    if print_debate:
        print(f"{role1} vs. {role2}\n Issue:{issue} \n\n")
        print(cum_arg)
        if adjudicator:
            print("\n\nADJUDICATION: " + adjudication.choices[0].message.content)
        if synthesizer:
            print("\n\nSYNTHESIS: " + synthesis.choices[0].message.content)

              
    return cum_arg,adjudication,synthesis
#star_trekwars = debateGPT("STAR TREK FAN","STAR WARS FAN","Which is superior, Star Trek or Star Wars?")
#print
#print(star_trekwars)

In [12]:
def position_generator(issue,no_positions=2,print_positions = False,model="gpt-3.5-turbo"):
    '''Produces no_positions positions on issue'''
    position_generator_prompt = f"Generate {no_positions} different positions on {issue}.  Be brief."
    positions = openai.ChatCompletion.create(
          model=model,
          messages=[
            {"role": "user", "content": position_generator_prompt}
              ]
            )
    if print_positions:
        print("\n\nPOSITIONS: " + positions.choices[0].message.content)
        
    return positions

position_generator(issue,print_positions=True);
    
    
    



POSITIONS: Position 1: The best foundation for ethics is religion
- Many religious beliefs provide a moral framework and ethical guidelines for individuals to follow.
- Religion often emphasizes concepts such as the golden rule, promoting kindness, justice, and compassion.
- The teachings and principles of religion can provide a strong foundation for ethical decision-making.

Position 2: The best foundation for ethics is a secular humanistic approach
- Secular humanism focuses on human values, ethics, and reason without relying on religious or supernatural beliefs.
- It emphasizes the importance of human well-being, rationality, and critical thinking in ethical decision-making.
- A secular humanistic approach brings people together regardless of their religious beliefs and provides a common ground for ethical discussions and debates.


In [5]:
role1 = "KANTIAN ETHICIST"
role2 = "UTILITARIAN ETHICIST"

message_adjective = "concisely"
issue = "What is the best foundation for ethics?"
debateGPT(role1,role2,issue,synthesizer = True);

KANTIAN ETHICIST vs. UTILITARIAN ETHICIST
 Issue:What is the best foundation for ethics? 


KANTIAN ETHICIST: As a Kantian ethicist, I firmly believe that the best foundation for ethics lies in the concept of universal moral duties and principles. Ethics should be grounded in reason and rationality rather than solely on maximizing overall happiness or utility, as advocated by utilitarianism.

Kantian ethics emphasizes the importance of autonomy, rationality, and respect for the inherent dignity of every individual. The foundation of ethics should be based on the categorical imperative, which states that an action is morally right if and only if it can be willed as a universal law without contradiction. This principle highlights the importance of moral duties that are binding for all individuals, irrespective of personal desires or the consequences of their actions.

Unlike utilitarian ethics, which focuses on the outcomes or consequences of actions, Kantian ethics places value on the i

In [5]:
debateGPT("Proponent that the egg came before the chicken ","Proponent that the chicken came before the egg","Which came first the chicken or the egg?");

Proponent that the egg came before the chicken  vs. Proponent that the chicken came before the egg on Which came first the chicken or the egg? 


Proponent that the egg came before the chicken : I firmly believe that the egg came before the chicken. The process of evolution teaches us that species gradually evolve over time through genetic mutations and natural selection. Therefore, at some point in history, a pre-chicken species laid an egg containing the genetic mutation that led to the first chicken. Hence, the egg came first in the evolutionary chain, ultimately giving rise to the chicken.

Proponent that the chicken came before the egg: I firmly believe that the chicken came before the egg. From a scientific standpoint, we can trace the origin of chickens through gradual evolutionary processes. These processes involve the mating and reproduction of two pre-chicken ancestors, which eventually led to the formation of the first chicken. Therefore, the chicken must have existed first 

In [10]:
debateGPT("Proponent that the egg came before the chicken ","Proponent that the chicken came before the egg","Which came first the chicken or the egg?",synthesizer = True);

Proponent that the egg came before the chicken  vs. Proponent that the chicken came before the egg
 Issue:Which came first the chicken or the egg? 


Proponent that the egg came before the chicken : I firmly advocate that the egg came before the chicken.

Proponent that the chicken came before the egg: I firmly believe that the chicken came before the egg. The logic is simple: for an egg to exist, it needs to be laid by a fully formed chicken. The process of evolution dictates that species evolve gradually over time. Thus, it is reasonable to assert that a pre-chicken species gradually evolved into the first chicken through genetic mutations. Therefore, the chicken had to come first, before the egg could exist.

Proponent that the egg came before the chicken : Proponent that the egg came before the chicken: While your logic is understandable, it is important to consider the concept of speciation and the role of mutations in the evolutionary process. Evolutionary biologists argue that n